# How to find answers using jupyter?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?


How to sanity check the environment?

In [2]:
var vm = require('vm');
var fs = require('fs');
// TODO: exports, require, module, __filename, __dirname
if (typeof importNotebook == 'undefined') {
    require('child_process').execSync('npm install JSONStream');
}
$$.done('import ready');


How to import a cells from jupyter?

In [2]:
if (typeof runningNotebooks == 'undefined') {
    var runningNotebooks = [];
}
var getCells = (notebook) => {
    var cellsToRun = [];
    if (runningNotebooks.indexOf(notebook) > -1) {
        if (notebook.indexOf('How to find answers') > -1) {
            throw 'Successfully imported How to find answers';
        } else {
            throw 'Notebook recursion detected ' + runningNotebooks.toString();
        }
    }
    runningNotebooks[runningNotebooks.length] = notebook;
    var cells = fs.createReadStream(notebook).pipe(require('JSONStream').parse('cells.*'));
    cells.on('data', (cell) => {
        if(cell.cell_type === 'code') {
            cellsToRun[cellsToRun.length] = cell.source.join('').trim();
        }
    });
    return new Promise((resolve, reject) => {
        cells.on('error', e => reject(e));
        cells.on('close', () => resolve(cellsToRun))
    });
};
$$.done(getCells);


How to create VM promises out of the cell content?

In [2]:
var promises = (cellsToRun) => {
    var context = vm.createContext(Object.assign({}, global));
    var script = new vm.Script('() => (vm, code) => vm.runInThisContext(code)');
    var myVM = script.runInContext(context)();
    //return ((myVM, context) => {
    return cellsToRun.map(cell => {
        return new Promise((resolve, reject) => {
            var result;
            try {
                result = myVM(vm, cell);
                console.log(result);
                if (!context.$$._isAsync) {resolve(result)}
            } catch (e) {
                resolve(e);
            }
        });
    });
    //})(myVM, context);
};
$$.done(promises);


How to resolve a list of promises?

How to execute all the cells in a notebook?



In [2]:

var runAllCells = (cellsToRun) => {
    var p = promises(cellsToRun);
    var results = [];
    return p.reduce((p, result) => {
        results[results.length] = result;
        return p.then(() => {
            console.log(result);
            return result;
        });
    },Promise.resolve()).then((results) => {
        var index = runningNotebooks.indexOf(notebook);
        if (index > -1) {
            runningNotebooks.splice(index, 1);
        }
        return results;
    }).catch(e => done(e));
};
$$.done(runAllCells);


In [6]:
var importNotebook = (notebook) => {
    if(typeof notebook !== 'undefined') {
        return getCells(notebook).then((cells) => {
            return runAllCells(cells).then((cells) => {
                console.log('executed notebook ' + notebook);
                return cells;
            });
        });
    }
}
$$.done(importNotebook);



How to provide this command as a service to other notebooks?

What to include to create inheritence in jupyter?

How to turn jupyter notebooks in to modules?



In [1]:
// Does this work everywhere?
if (typeof notebook == 'undefined') {
    var notebook = 'How to find answers.ipynb';
}
if (typeof importNotebook == 'undefined') {
    importNotebook = true;
    var co = '', vm = require('vm'),
        source = require('fs').readFileSync(notebook).toString(),
        m, re = (/"cell_type": "code",\s*"source": \[\s([\s\S]*?)\s*\]\s*,\s*/ig);
    while (m = re.exec(source)) { co += (co == '' ? '' : ',"\\n",\n' ) + m[1]; }
    co = '(function ($$, require, process) {\n\n' + JSON.parse('['+co+']').join('') + '\n\n; $$.done(importNotebook); return importNotebook; })';
    var context = vm.createContext(Object.assign({}, global));
    context.global = context;
    context.$$ = Object.assign({}, $$);
    Object.assign(context.$$, {async: () => {}, done: (done) => console.log(done)});
    importNotebook = vm.runInThisContext(co)(context.$$, require, process);
    // test to make sure it works
    /*
    $$.async();
    importNotebook(notebook).then((done) => {
        console.log(done);
        $$.mime({'text/markdown' : 'importNotebook() usage:\n\n```\nimportNotebook(path: string);\n```'});
        $$.done(done);
    });
    */
}


These are some other useful notebooks you may want to import.

How to train a chatbot using jupyter services?


In [2]:
/*
$$.async();
importNotebook('How to use express.ipynb').then((done) => {
    $$.done(done);
}).catch(e => $$.done(e));
*/



In [2]:
console.log('hit')